In [10]:
import pandas as pd
import csv
import json
import nltk
import numpy as np
from pandas.io.json import json_normalize

file = "C:\\Users\\user\\Desktop\\semantics.json"
file1 = "C:\\Users\\user\\Desktop\\reviews1.json"  #reviews1.json
df = pd.read_json(file1,typ='series')
df_rev = json_normalize(df['Reviews'])
df_rev['full_content'] = df_rev['Content'] + '. ' + df_rev['Title']
pos = pd.read_json(file,typ='series')

newcol = df_rev['full_content']
newcol.to_csv('content_reviews.csv',header=False, index=False, encoding='utf-8')


0      Hotel is clean and very close to airport. Staf...
1      Very convenient for LAX (free shuttle). New ve...
2      The thing I like most about Holiday Inn LAX is...
3      We stayed here one night before taking a cruis...
4      This hotel is just minutes away from LAX with ...
5      I stay at this hotel a lot. The staff is nice ...
6      This hotel is good for a one night stay close ...
7      I've been trying to call the hotel for the pas...
8      It's a hit &the miss one week go od another ba...
9      Marginal hotel, but close in proximity to LAX....
10     We stayed in this hotel as we transited to & f...
11     I'd recommend this hotel for a quick stopover ...
12     Check in was quick and easy, our room was big ...
13     I stayed at this hotel for 1 night prior to fl...
14     Concierge and staff were very friendly, room w...
15     Even as I write this I find it hard to describ...
16     Overall, we had an average experience at this ...
17     My wife is an airline em